In [3]:
import os
import hashlib
import pandas as pd
import numpy as np
from collections import Counter
import math, statistics

csv_path = os.path.join(os.curdir, "datasets", "dataset - Privacy-Engineering.csv")
zip_path = os.path.join(os.curdir, "datasets", "matched_zips.csv")

df = pd.read_csv(csv_path)
df_zip = pd.read_csv(zip_path, dtype={"zip": 'string'})
matched_zips = df_zip.set_index('zip')['state'].to_dict()

Drop useless columns
---

In [4]:
columns_to_drop = ["ancestry"]
df.drop(columns_to_drop, axis=1, inplace=True)
df.head(5)

,id,gender,dob,zipcode,education,employment,children,marital_status,number_vehicles,commute_time,accommodation,disease
0,Amelia Morrow,female,12/1950,26904,High School,Retired,2,married,2,0.0,Own house,Alzheimer's disease
1,Lillian Cole,female,4/1940,78088,PhD/md,Retired,1,married,1,0.0,Rent flat,multiple sclerosis
2,Lillian Simpson,female,12/1966,51502,Bachelor,Employed,1,married,0,0.1,Rent flat,heart disease
3,Avery Richards,female,4/1943,54080,Bachelor,Retired,1,married,0,0.0,Rent room,endometriosis
4,Sophia Alvarado,female,10/1947,68785,High School,Retired,1,married,0,0.0,Rent flat,gastritis


Generate a unique id based on the hash of the initial id, the date of birth and the zipcode
---

In [5]:
salt = b"\x91\xff=s\x18?JH\x940\xf1$wN\xa4\xbf\x9f\xcdi\x8c\xa2\xf3\xe3I\xeb\xf4\x1f\xc0y\x94\x84\xa8"

df.id = df.apply(lambda x : hashlib.sha512((str(x.id) + str(x.dob) + str(x.zipcode)).encode() + salt).hexdigest(), axis=1)
df.head(5)

,id,gender,dob,zipcode,education,employment,children,marital_status,number_vehicles,commute_time,accommodation,disease
0,8f6c050946988234e80586775ff401a86898b270b5111a...,female,12/1950,26904,High School,Retired,2,married,2,0.0,Own house,Alzheimer's disease
1,43ae9c63fa40fa3e4873f50b19730c670808cb78f789b2...,female,4/1940,78088,PhD/md,Retired,1,married,1,0.0,Rent flat,multiple sclerosis
2,ac937a096e35bb40f0776d0d3505a32168baf853353865...,female,12/1966,51502,Bachelor,Employed,1,married,0,0.1,Rent flat,heart disease
3,543f72d9aa3fed3bfe35c324151a2ab9bf5e570ef56293...,female,4/1943,54080,Bachelor,Retired,1,married,0,0.0,Rent room,endometriosis
4,ef018d17c59cea78a0eecc1d8ee6043236c5438f070e73...,female,10/1947,68785,High School,Retired,1,married,0,0.0,Rent flat,gastritis


In [6]:
def find_state(zip_code):
    res = matched_zips.get(str(zip_code)[0:3])
    if res is None: return "Unknown"
    return res

df['state'] = df.zipcode.apply(find_state)
df = df.drop("zipcode", axis=1)
df.head(5)

,id,gender,dob,education,employment,children,marital_status,number_vehicles,commute_time,accommodation,disease,state
0,8f6c050946988234e80586775ff401a86898b270b5111a...,female,12/1950,High School,Retired,2,married,2,0.0,Own house,Alzheimer's disease,Unknown
1,43ae9c63fa40fa3e4873f50b19730c670808cb78f789b2...,female,4/1940,PhD/md,Retired,1,married,1,0.0,Rent flat,multiple sclerosis,(TX) Texas
2,ac937a096e35bb40f0776d0d3505a32168baf853353865...,female,12/1966,Bachelor,Employed,1,married,0,0.1,Rent flat,heart disease,(IA) Iowa
3,543f72d9aa3fed3bfe35c324151a2ab9bf5e570ef56293...,female,4/1943,Bachelor,Retired,1,married,0,0.0,Rent room,endometriosis,(WI) Wisconsin
4,ef018d17c59cea78a0eecc1d8ee6043236c5438f070e73...,female,10/1947,High School,Retired,1,married,0,0.0,Rent flat,gastritis,(NE) Nebraska


In [7]:


def more_than_2_children(x):
    if isinstance(x, str): return x
    elif x == 0: return '0'
    elif x == 1: return '1'
    elif x > 1: return '2+'

def has_child(childs):
    if childs == 0: return False
    return True

def more_than_1_hour(x):
    if isinstance(x, str): return x
    elif x == 0.0: return '0'
    elif x <= 1.0: return '[0-1]'
    elif x > 1.0: return '1+'

def change_accom(accomod):
    how, _ = accomod.split(' ')
    return how

def change_education(educ):
    if educ == "PhD/md" or educ == "Masters": return "Masters and higher"
    return educ

def change_dob(dob):
    _, year = dob.split('/')
    year = int(year)
    while year % 3 != 0: year -= 1
    return f"[{year} - {year+2}]"

df.dob = df.dob.apply(change_dob)

df.accommodation = df.accommodation.apply(change_accom)
df.education = df.education.apply(change_education)
# df.children=df.children.apply(more_than_2_children)

df.children=df.children.apply(has_child)

df.commute_time=df.commute_time.apply(more_than_1_hour)

df.head(5)

,id,gender,dob,education,employment,children,marital_status,number_vehicles,commute_time,accommodation,disease,state
0,8f6c050946988234e80586775ff401a86898b270b5111a...,female,[1950 - 1952],High School,Retired,True,married,2,0,Own,Alzheimer's disease,Unknown
1,43ae9c63fa40fa3e4873f50b19730c670808cb78f789b2...,female,[1938 - 1940],Masters and higher,Retired,True,married,1,0,Rent,multiple sclerosis,(TX) Texas
2,ac937a096e35bb40f0776d0d3505a32168baf853353865...,female,[1965 - 1967],Bachelor,Employed,True,married,0,[0-1],Rent,heart disease,(IA) Iowa
3,543f72d9aa3fed3bfe35c324151a2ab9bf5e570ef56293...,female,[1941 - 1943],Bachelor,Retired,True,married,0,0,Rent,endometriosis,(WI) Wisconsin
4,ef018d17c59cea78a0eecc1d8ee6043236c5438f070e73...,female,[1947 - 1949],High School,Retired,True,married,0,0,Rent,gastritis,(NE) Nebraska


In [8]:
def entropy(C, N=2000): return round(- sum([(ci/N) * math.log(ci/N, 2) for i,ci in enumerate(C)]),4)
# Max : 11

In [9]:
from itertools import *
from tqdm import tqdm


covariates = ["education", "gender", "dob", "state", "children", "employment", "marital_status", "number_vehicles", "commute_time", "accommodation"]

results = []

for c in tqdm(range(1, len(covariates)+1)):
    for used in combinations(covariates, c):
        used = used + ("disease",)
        used = list(used)
        ct = df.groupby(used).size().to_frame("count")
        len_groups = len(ct)
        len_unique = sum(ct['count'] < 2)

        C = ct['count'].to_numpy()
        CR = C[C > 1]

        size_cr = 2000-len_unique

        entropy_full = entropy(C)
        entropy_cut = entropy(CR, size_cr)

        results.append(
            {
                "used": used,
                "len_groups": len_groups,
                "len_unique": len_unique,
                "diff": size_cr,
                "entropy_full": entropy_full,
                "entropy_cut": entropy_cut
            }
        )

100%|██████████| 10/10 [00:05<00:00,  1.82it/s]


In [10]:
results_sort = sorted(results, key=lambda x: (x['diff'], x['len_groups'],  x['entropy_cut'], x['entropy_full'], ), reverse=True)

In [11]:
results_filter = [ dic for dic in results_sort if dic.get('len_unique') > 0 and dic.get('len_unique') < 100 and dic.get('entropy_full') > 5]    
results_filter_sorted = sorted(results_filter, key=lambda x: x['entropy_cut'], reverse=True)

In [12]:
# Best ?
results_filter_sorted[:5]

[{'used': ['gender', 'dob', 'disease'],
  'len_groups': 533,
  'len_unique': 80,
  'diff': 1920,
  'entropy_full': 8.8033,
  'entropy_cut': 8.6543},
 {'used': ['education',
   'gender',
   'number_vehicles',
   'accommodation',
   'disease'],
  'len_groups': 430,
  'len_unique': 94,
  'diff': 1906,
  'entropy_full': 8.3062,
  'entropy_cut': 8.1056},
 {'used': ['dob', 'children', 'disease'],
  'len_groups': 394,
  'len_unique': 90,
  'diff': 1910,
  'entropy_full': 8.2104,
  'entropy_cut': 8.0142},
 {'used': ['education', 'number_vehicles', 'commute_time', 'disease'],
  'len_groups': 392,
  'len_unique': 76,
  'diff': 1924,
  'entropy_full': 8.1299,
  'entropy_cut': 7.962},
 {'used': ['dob', 'disease'],
  'len_groups': 304,
  'len_unique': 16,
  'diff': 1984,
  'entropy_full': 7.9867,
  'entropy_cut': 7.9511}]

In [13]:
g = ["education", "gender", "number_vehicles", "accommodation", "employment", "children", "commute_time"]

# Demandez pas pourquoi les noms sont ct..
ctt = df[df.duplicated(g, keep=False)].sort_values(by=g) # Enlève ceux qui sont uniques (sans disease pris en compte)

g2 = g.copy()
g2.append("disease")

g3 = []
cttt = ctt.groupby(g2).size().to_frame("count") # disease inclus


def checkForUnique(grouped_frame):
    checked = {}    
    for i,row in enumerate(grouped_frame.iterrows()):
        *identifiers, _ = row[0] 
        identifiers = ':'.join([str(iden) for iden in identifiers])
        if identifiers in checked:
            if checked.get(identifiers).get("found") is True: continue
            else : checked[identifiers]["found"] = True
        else: checked[identifiers] = {"found": False, "index": i+2}
    return checked

false_checked = [ v.get('index') for k,v in checkForUnique(cttt).items() if v.get("found") == False]
print(false_checked)

g_final = ["id", "gender", "education", "employment", "children", "number_vehicles", "commute_time", "accommodation", "disease"]
base_1 = ctt[g_final].reset_index().drop(index=false_checked) # enlève ceux qui sont pas bon pour le l-diversity
base_1 = base_1.drop(columns=["index"])

base_1.__len__()

[106, 202, 227, 873, 917]


1857

In [14]:
g = ["education", "gender", "number_vehicles", "accommodation", "employment", "children", "commute_time"]
gbqi = df[g+["disease"]].groupby(g)
indexes_to_drop = gbqi.filter(lambda x:x.disease.nunique()==1).index
df[["id"]+g+["disease"]].drop(indexes_to_drop).to_csv("FINAL.csv", index=False)

In [45]:
d = pd.DataFrame(
    dict(
        a = ["coucou", "papa", "coucou", "papa"],
        b = ["papa", "papa", "coucou", "papa"],
        c = [0, 1, 1, 2]
    )
)

d.groupby(['a','b']).c.nunique()
#d.groupby(['a','b']).c.count()

a       b     
coucou  coucou    1
        papa      1
papa    papa      2
Name: c, dtype: int64